<a href="https://colab.research.google.com/github/cailuyu/dust3r/blob/main/dust3r_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup environment - this should take ~ 2 min.

In [ ]:
#Avoid cell growing like crazy in setup
from google.colab.output import eval_js
eval_js('google.colab.output.setIframeHeight("250")')
import os

#Clone Repo
%cd /content
!git clone --recursive https://github.com/naver/dust3r
%cd /content/dust3r

#Setup Environment
!pip install -r requirements.txt

if not os.path.exists('checkpoints'):
    #Download weights
    !mkdir -p checkpoints/
    !wget https://download.europe.naverlabs.com/ComputerVision/DUSt3R/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth -P checkpoints/

    #Compile curope -- This takes too much time, uncomment to compile it (in think its not worth in colab mode)
    # %cd /content/dust3r/croco/models/curope/
    # !python setup.py build_ext --inplace

/content
Cloning into 'dust3r'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 69 (delta 10), reused 66 (delta 8), pack-reused 0
Receiving objects: 100% (69/69), 567.59 KiB | 14.19 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Submodule 'croco' (https://github.com/naver/croco) registered for path 'croco'
Cloning into '/content/dust3r/croco'...
remote: Enumerating objects: 119, done.        
remote: Counting objects: 100% (119/119), done.        
remote: Compressing objects: 100% (84/84), done.        
remote: Total 119 (delta 47), reused 98 (delta 30), pack-reused 0        
Receiving objects: 100% (119/119), 383.08 KiB | 11.61 MiB/s, done.
Resolving deltas: 100% (47/47), done.
Submodule path 'croco': checked out '743ee71a2a9bf57cea6832a9064a70a0597fcfcb'
/content/dust3r
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

## Run demo (this will create a shareable link with gradio)

In [ ]:
%cd /content/dust3r

#Simulate were running the original demo.py but in colab cell
#!python3 demo.py --weights checkpoints/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth

#Change default weights and gradio running mode with inline editing
!sed -i 's/parser.add_argument("--weights", type=str, required=True, help="path to the model weights")/parser.add_argument("--weights", type=str, default='"'"'\/content\/dust3r\/checkpoints\/DUSt3R_ViTLarge_BaseDecoder_512_dpt.pth'"'"', help="path to the model weights")/' demo.py
!sed -i 's/demo.launch(share=False, server_name=server_name, server_port=server_port)/demo.launch(inline=True, debug=True)/' demo.py

from demo import *

#Dirty'n'quick trick to allow parse_args to work in colab cell without exploding
import sys; sys.argv = sys.argv[2:]

parser = get_args_parser()
args = parser.parse_args()

if args.tmp_dir is not None:
    tmp_path = args.tmp_dir
    os.makedirs(tmp_path, exist_ok=True)
    tempfile.tempdir = tmp_path

if args.server_name is not None:
    server_name = args.server_name
else:
    server_name = '0.0.0.0' if args.local_network else '127.0.0.1'

model = load_model(args.weights, args.device)
# dust3r will write the 3D model inside tmpdirname
with tempfile.TemporaryDirectory(suffix='dust3r_gradio_demo') as tmpdirname:
    print('Outputing stuff in', tmpdirname)
    main_demo(tmpdirname, model, args.device, args.image_size, server_name, args.server_port)